# Retrieving data via bitcoin-cli

In [1]:
# install necessary python libraries 
#%pip install python-bitcoinrpc
#%pip install paramiko

In [2]:
# read credentials from json file 
import pandas as pd

credentials = pd.read_json('credentials.json')

In [3]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

## Step 1: Monitor newly incoming files 

In [4]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblockchaininfo") # returns the header hash of the most recent block on the best block chain.

In [5]:
# parse json output
import json
blockchaininfo = json.loads(stdout.read().decode("utf-8"))

bestblockhash = blockchaininfo['bestblockhash']
headers = blockchaininfo['headers']
print(bestblockhash)
print(headers)

000000000000000000099901fbaa4983ca0f7792b73acab7db6578282d57569f
683070


## Step 2: Get information about the new block

In [6]:
# get blockinfo 
command = "bitcoin-cli getblock " + bestblockhash + " 2"
stdin, stdout, stderr = ssh.exec_command(command)

block = json.loads(stdout.read().decode("utf-8"))

from datetime import datetime, timezone
ts_epoch = block['time']
block_timestamp = datetime.fromtimestamp(ts_epoch)
block_date = block_timestamp.strftime('%Y-%m-%d')
block_hash = block['hash']
block_height = block['height']
previousblockhash = block['previousblockhash']
# nextblockhash = block['nextblockhash']

# build json object
data = {}
data['block_hash'] = block_hash
data['block_height'] = block_height
data['block_timestamp'] = str(block_timestamp)
data['block_date'] = str(block_date)
data['previousblockhash'] = previousblockhash # relevant for PRECEDES relationship
# data['nextblockhash'] = nextblockhash
json_data = json.dumps(data, indent=4, sort_keys=False)

print(json_data)


{
    "block_hash": "000000000000000000099901fbaa4983ca0f7792b73acab7db6578282d57569f",
    "block_height": 683070,
    "block_timestamp": "2021-05-11 09:54:50",
    "block_date": "2021-05-11",
    "previousblockhash": "000000000000000000035a715d554ed41fdd029bf89cb0f8c0ea1027efc28511"
}


## Step 3: Iterate through the list of txids and retrieve information on tx

In [7]:
###  parse transactions which are contained in one block 

# Testing section 
#  - saving a few transactions in list
#  - to prevent from parsing all transactions, set parse_all to False
parse_all = True 
c = 0
txs_needed = 1
offset = 0
txs = []


# timing of transaction retrieval for one block
import time
startTime = time.time() # measure execution time


# start of parsing !!!! using [1:] to skip coinbase !!!!!
for tx in block['tx'][offset:]: 
    command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    rawtx = json.loads(stdout.read().decode("utf-8"))
    
    address_list = []  

    for o in rawtx['vout']: 
        # check whether there are output adresses
        if 'addresses' in o['scriptPubKey']:
            for a in o['scriptPubKey']['addresses']: 
                # a Python object (dict):
                addrObj = {
                   "addr": a,
                    "output_nr": o['n']
                }
                jAddr = json.dumps(addrObj)
                # convert json string to Json Dict
                jsonDict = json.loads(jAddr)
                address_list.append(jsonDict)
        
    # print json object = one transaction within a block
    txdata = {}
    txdata['txid'] = tx['txid']
    txdata['block_hash'] = block_hash
    txdata['block_date'] = str(block_date)
    txdata['address_list'] = address_list
    txdata['inDegree'] = len(rawtx['vin']) 
    txdata['outDegree'] = len(rawtx['vout'])
    
    # creating input list  ## Throwing error for 
    input_list = []
    for i in rawtx['vin']: 
        # bypassing coinbase transactions!
        try:
            addrObj = {
               "txid": i['txid'],
                "output_nr": i['vout']
            }
            jAddr = json.dumps(addrObj)
            jsonDict = json.loads(jAddr)
            input_list.append(jsonDict)
        
            txdata['input_list'] = input_list
        
        # reformulate and handle coinbase!
        except: pass
    
    tx_json_data = json.dumps(txdata, indent=4, sort_keys=False)
    
    
    # Testing by sample creation
    if parse_all == False:
        txs.append(tx_json_data)
        c += 1
        if c >= txs_needed:
            break

    #print(tx_json_data)

    
display(rawtx)
print(tx_json_data)    
    
        
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

{'txid': '89396ac12db5327463777292228eedb9b9ffcd5afde8d1b829346ad7a67f3358',
 'hash': '6cb540fac775a3d6099615c79f2ea91ab21330e3fb0da03d2ec3c5a0f3372df6',
 'version': 1,
 'size': 246,
 'vsize': 164,
 'weight': 654,
 'locktime': 0,
 'vin': [{'txid': '1200df64074fb7fce4b40a1f784309a3d1d33fa413cb1df2aeca692d78565468',
   'vout': 3,
   'scriptSig': {'asm': '00144f6ffebd3dffc5613c334666f24bfe19b47a857a',
    'hex': '1600144f6ffebd3dffc5613c334666f24bfe19b47a857a'},
   'txinwitness': ['3045022100ab4c3204b23d716fe9f0b35a4b71c786d490822eaf6a6d28bf9ae4f6a998116302203922de278ee736b08d84b50b9859bc9614f087bb4e4967b1f3e560121dbef74a01',
    '0311aa8a4c72d2fa87bbd0fbfed71618a99535f0f2c8827cf5cff062d740dd9386'],
   'sequence': 4294967295}],
 'vout': [{'value': 0.02577948,
   'n': 0,
   'scriptPubKey': {'asm': '0 0d7e3778ee29e9b036347dd2fbebfa424fe4254d',
    'hex': '00140d7e3778ee29e9b036347dd2fbebfa424fe4254d',
    'reqSigs': 1,
    'type': 'witness_v0_keyhash',
    'addresses': ['bc1qp4lrw78w985mqd3

{
    "txid": "89396ac12db5327463777292228eedb9b9ffcd5afde8d1b829346ad7a67f3358",
    "block_hash": "000000000000000000099901fbaa4983ca0f7792b73acab7db6578282d57569f",
    "block_date": "2021-05-11",
    "address_list": [
        {
            "addr": "bc1qp4lrw78w985mqd350hf0h6l6gf87gf2d2hpq0g",
            "output_nr": 0
        },
        {
            "addr": "bc1qdv869a0zsas6zzleupepzndpgtd037ttxz03l2",
            "output_nr": 1
        }
    ],
    "inDegree": 1,
    "outDegree": 2,
    "input_list": [
        {
            "txid": "1200df64074fb7fce4b40a1f784309a3d1d33fa413cb1df2aeca692d78565468",
            "output_nr": 3
        }
    ]
}
Execution time in seconds: 104.1369400024414


## General information Retrieval Test

In [8]:
command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
stdin, stdout, stderr = ssh.exec_command(command)
rawtx = json.loads(stdout.read().decode("utf-8"))
print(rawtx)

{'txid': '89396ac12db5327463777292228eedb9b9ffcd5afde8d1b829346ad7a67f3358', 'hash': '6cb540fac775a3d6099615c79f2ea91ab21330e3fb0da03d2ec3c5a0f3372df6', 'version': 1, 'size': 246, 'vsize': 164, 'weight': 654, 'locktime': 0, 'vin': [{'txid': '1200df64074fb7fce4b40a1f784309a3d1d33fa413cb1df2aeca692d78565468', 'vout': 3, 'scriptSig': {'asm': '00144f6ffebd3dffc5613c334666f24bfe19b47a857a', 'hex': '1600144f6ffebd3dffc5613c334666f24bfe19b47a857a'}, 'txinwitness': ['3045022100ab4c3204b23d716fe9f0b35a4b71c786d490822eaf6a6d28bf9ae4f6a998116302203922de278ee736b08d84b50b9859bc9614f087bb4e4967b1f3e560121dbef74a01', '0311aa8a4c72d2fa87bbd0fbfed71618a99535f0f2c8827cf5cff062d740dd9386'], 'sequence': 4294967295}], 'vout': [{'value': 0.02577948, 'n': 0, 'scriptPubKey': {'asm': '0 0d7e3778ee29e9b036347dd2fbebfa424fe4254d', 'hex': '00140d7e3778ee29e9b036347dd2fbebfa424fe4254d', 'reqSigs': 1, 'type': 'witness_v0_keyhash', 'addresses': ['bc1qp4lrw78w985mqd350hf0h6l6gf87gf2d2hpq0g']}}, {'value': 0.54321, 'n

In [9]:
test_list = []
test_list.append('1')
print(test_list)

['1']


In [10]:
test_list = []
# a Python object (dict):
x = {
  "name": "John",
  "age": 30,
  "city": "New York"
}

# convert into JSON string:
y = json.dumps(x)
# convert json string to Json Dict
jsonDict = json.loads(y)
test_list.append(jsonDict)

print(test_list)

[{'name': 'John', 'age': 30, 'city': 'New York'}]


## Kafka Producer



In [ ]:
%pip install kafka-python

#### Example Blocks

In [ ]:
blocks = [ \
{
    "block_hash": "0000000000000000000be676cdc68e8f4b7f9a60743f6aa4b63cb8cbbdb9154d",
    "block_height": 682948,
    "block_timestamp": "2021-05-10 17:16:37",
    "block_date": "2021-05-10",
    "previousblockhash": "0000000000000000000070815fbd03d22c4914cfb0348fbbea3b7f878864a748"
},

{
    "block_hash": "0000000000000000000130c1c5607bc663835e8ce53ad99d7a9e67c8a99506f0",
    "block_height": 682949,
    "block_timestamp": "2021-05-10 17:34:30",
    "block_date": "2021-05-10",
    "previousblockhash": "0000000000000000000be676cdc68e8f4b7f9a60743f6aa4b63cb8cbbdb9154d"
},

{
    "block_hash": "000000000000000000018c81e5c4ce50114eab530aa3a7c01d4dc171fa16bf03",
    "block_height": 682950,
    "block_timestamp": "2021-05-10 17:45:30",
    "block_date": "2021-05-10",
    "previousblockhash": "0000000000000000000130c1c5607bc663835e8ce53ad99d7a9e67c8a99506f0"
},

{
    "block_hash": "00000000000000000005aeb509abd1c98af7db195a6e57898e1273e7a50eed51",
    "block_height": 682951,
    "block_timestamp": "2021-05-10 17:45:57",
    "block_date": "2021-05-10",
    "previousblockhash": "000000000000000000018c81e5c4ce50114eab530aa3a7c01d4dc171fa16bf03"
},

{
    "block_hash": "000000000000000000056e16b2d1d46e0370f0277bb7b62dfb88db2a2e400d8c",
    "block_height": 682952,
    "block_timestamp": "2021-05-10 17:52:50",
    "block_date": "2021-05-10",
    "previousblockhash": "00000000000000000005aeb509abd1c98af7db195a6e57898e1273e7a50eed51"
},

{
    "block_hash": "00000000000000000003c54646926bcdafa0bd3fe194564cbca2e44d1ac7bb4f",
    "block_height": 682953,
    "block_timestamp": "2021-05-10 17:57:14",
    "block_date": "2021-05-10",
    "previousblockhash": "000000000000000000056e16b2d1d46e0370f0277bb7b62dfb88db2a2e400d8c"
},

{
    "block_hash": "0000000000000000000d9d791740148bc2e80501219697399bafa73a12717c5a",
    "block_height": 682954,
    "block_timestamp": "2021-05-10 17:57:14",
    "block_date": "2021-05-10",
    "previousblockhash": "00000000000000000003c54646926bcdafa0bd3fe194564cbca2e44d1ac7bb4f"
},

{
    "block_hash": "0000000000000000000483beb70ad8fd8475da0cd0417499c5a310ef6ddb3561",
    "block_height": 682955,
    "block_timestamp": "2021-05-10 17:59:06",
    "block_date": "2021-05-10",
    "previousblockhash": "0000000000000000000d9d791740148bc2e80501219697399bafa73a12717c5a"
}]

#### Pushing them to the "blocks" topic

In [ ]:
import json
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers='localhost:9092')
producer = KafkaProducer(value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [ ]:
# Block test
for i in range(len(blocks)):
    producer.send('blocks', blocks[i])